In [31]:
from pathlib import Path
FILE = Path("file3.lol")

raw = FILE.read_bytes()
head = raw[:1024]

# imprimir hexdump
def ascii_preview(b, length=512):
  s = b[:length]
  txt = ''.join(chr(x) if 32 <= x < 127 or x in (9,10,13) else '.' for x in s)
  return txt

print("=== ASCII preview")
print(ascii_preview(head, 512))
print("\n=== Hex ===")
print(' '.join(f"{x:02x}" for x in head[:128]))

# comprobaciones sintácticas simples para Base64
B64_STD = set(b"ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/=")
# permite \n y carriage return en texto base64
B64_CHARS_WITH_NL = B64_STD.union({10,13})

# 1) bytes del head en base 64
only_b64 = all((c in B64_CHARS_WITH_NL) for c in head)
print("\n1) Bytes de head en b64?", only_b64)

# 2) padding con =
has_eq = b'=' in raw[-16:] or b'=\n' in raw or raw.rstrip().endswith(b'=')
print("2) Padding con =:", has_eq)

# 3) contar caracteres en ASCII
visible_chars = [c for c in head if 32 <= c < 127 or c in (9,10,13)]
if visible_chars:
    b64_count = sum(1 for c in visible_chars if c in B64_STD or c in (10,13))
    frac = b64_count / len(visible_chars)
else:
    frac = 0.0
print(f"3) Fraccion de contenido que es ASCII: {frac:.3f}")

# 4) Medir porcentaje de lineas longitud 4
lines = ascii_preview(raw, 4096).splitlines()
if lines:
    lengths = [len(l) for l in lines if len(l.strip())>0]
    if lengths:
        multiples_of_4 = sum(1 for L in lengths if L % 4 == 0)
        pct_mult4 = multiples_of_4 / len(lengths)
    else:
        pct_mult4 = 0.0
else:
    pct_mult4 = 0.0
print(f"4) Medir porcentaje de lineas longitud 4: {pct_mult4:.3f}")

# 5) Buscar firmas de formatos binarios
signs = []
if raw.startswith(b"%PDF"):
    signs.append("PDF")
if raw.startswith(b"\x89PNG\r\n\x1a\n"):
    signs.append("PNG")
if raw.startswith(b"\xff\xd8\xff"):
    signs.append("JPEG")
if raw.startswith(b"PK\x03\x04"):
    signs.append("ZIP")
print("5) Firmas de formatos binarios en el inicio:", signs if signs else "False")


=== ASCII preview
AAAAIGZ0eXBpc29tAAACAGlzb21pc28yYXZjMW1wNDEAABLebW9vdgAAAGxtdmhkAAAAAAAAAAAAAAAAAAAD6AAAGCAAAQAAAQAAAAAAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAwAAC8F0cmFrAAAAXHRraGQAAAADAAAAAAAAAAAAAAABAAAAAAAAF3AAAAAAAAAAAAAAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAABAAAAAAZAAAAGQAAAAAAAkZWR0cwAAABxlbHN0AAAAAAAAAAEAABdwAAAAAAABAAAAAAs5bWRpYQAAACBtZGhkAAAAAAAAAAAAAAAAAAA8AAABaABVxAAAAAAALWhkbHIAAAAAAAAAAHZpZGUAAAAAAAAAAAAAAABWaWRlb0hhbmRsZXIAAAAK5G1pbmYAAAAUdm1o

=== Hex ===
41 41 41 41 49 47 5a 30 65 58 42 70 63 32 39 74 41 41 41 43 41 47 6c 7a 62 32 31 70 63 32 38 79 59 58 5a 6a 4d 57 31 77 4e 44 45 41 41 42 4c 65 62 57 39 76 64 67 41 41 41 47 78 74 64 6d 68 6b 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 44 36 41 41 41 47 43 41 41 41 51 41 41 41 51 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 45 41 41 41 41 41 41 41 41 41 41 41 41 41

1) Bytes de head en b64? True
2) Padding con =: True
3) Fraccion de con

In [32]:
from pathlib import Path # se usa para agregar el path del archivo en colab
import re # solo lo uso para limpiar pero no es estrictamente necesario

archivoCodificado = "file3.lol"
archivoDecodificado = "file3_decoded"

# ------------------ implementación Base64 ------------------
_b64chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
_b64rev = {c: i for i, c in enumerate(_b64chars)}

def base64_decode_from_scratch(s):
  """
  Decodificador base64 desde cero.
  Acepta s como str (puede contener espacios/newlines).
  Devuelve bytes decodificados.
  """
  # Mantener solo chars válidos + '='
  s = ''.join(ch for ch in s if ch in _b64chars or ch == '=')
  if not s:
      return b''
  # Si la longitud no es múltiplo de 4, rellenar con '='
  pad_len = (4 - (len(s) % 4)) % 4
  if pad_len:
      s += '=' * pad_len

  out = bytearray()
  for i in range(0, len(s), 4):
      block = s[i:i+4]
      vals = []
      pad = 0
      for ch in block:
          if ch == '=':
              vals.append(0)
              pad += 1
          else:
              # Asumir 'A' (valor 0) si char inválido
              vals.append(_b64rev.get(ch, 0))
      # recomponer 24 bits
      triple = (vals[0] << 18) | (vals[1] << 12) | (vals[2] << 6) | (vals[3])
      b1 = (triple >> 16) & 0xFF
      b2 = (triple >> 8) & 0xFF
      b3 = triple & 0xFF
      out.append(b1)
      if pad < 2:
          out.append(b2)
      if pad == 0:
          out.append(b3)
  return bytes(out)

# ------------------ detectar si es base64 ------------------
BASE64_CHARSET = set(b"ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/=\n\r")

def looks_like_base64(raw_bytes, threshold_printable=0.90):
  """
  Heuristica que checa la cantidad de caracteres imprimibles del dataset
  y dependiendo de la proporcion nos dice si esta en base 64
  """
  sample = raw_bytes[:4096]
  if not sample:
      return False
  # los valores imprimibles en ASCII son del 32 al 126, tab \n y el 13
  printable = sum(1 for b in sample if 32 <= b < 127 or b in (9,10,13))
  frac_print = printable / len(sample)
  in_b64 = sum(1 for b in sample if b in BASE64_CHARSET)
  frac_b64 = in_b64 / len(sample)
  return frac_print >= threshold_printable and frac_b64 >= 0.80

# ------------------ detección de magic bytes ------------------
def detect_magic_relaxed(b, head_size=4096):
  """
  Busca identificar el tipo de archivo basado en los magic bytes
  Le hice una modificacion para que busque en los primeros head_size
  bytes porque no estaba al principio
  """
  head = b[:head_size]

  # buscamos algunos magic bytes comunes
  if b"\x89PNG\r\n\x1a\n" in head:
      return ".png", "PNG"
  if b"%PDF" in head:
      return ".pdf", "PDF"
  if b"ftyp" in head:
      return ".mp4", "MP4 (ftyp)"
  if head.startswith(b"ID3") or head[:2] == b"\xff\xd8":
      if head.startswith(b"ID3"):
          return ".mp3", "MP3 (ID3)"
      if head[:2] == b"\xff\xd8":
          return ".jpg", "JPEG"
  if b"PK\x03\x04" in head:
      return ".zip", "ZIP (PK..)"
  if head[:6] in (b"GIF87a", b"GIF89a"):
      return ".gif", "GIF"
  return ".bin", "unknown"

# ------------------ pipeline ------------------
def process_file(in_path=archivoCodificado, archivoDecodificado=archivoDecodificado):
  p = Path(in_path)
  if not p.exists():
      raise FileNotFoundError(f"No existe {in_path} en el directorio actual ({Path.cwd()})")
  raw = p.read_bytes()
  print(f"[+] Leído {in_path}: {len(raw)} bytes")

  decoded_bytes = None

  if looks_like_base64(raw):
      print("[*] Heurística: parece Base64")
      s = raw.decode('ascii', errors='ignore')
      clean = re.sub(r'[^A-Za-z0-9+/=\n\r]', '', s)
      try:
          decoded_bytes = base64_decode_from_scratch(clean)
          print(f"[+] Decodificación Base64 OK, tamaño {len(decoded_bytes)} bytes")
      except Exception as e:
          print("[!] El decodificador propio falló:", e)
          decoded_bytes = None

  # detectar magic bytes
  ext, kind = detect_magic_relaxed(decoded_bytes)
  out_name = f"{archivoDecodificado}{ext}"
  Path(out_name).write_bytes(decoded_bytes)
  print(f"[+] Guardado: {out_name}  (detected: {kind})")

  return out_name

# Ejecutar pipeline
if __name__ == "__main__":
  outfile = process_file(archivoCodificado, archivoDecodificado)
  print("Salida:", outfile)
  try:
      from IPython.display import display, Video
      if outfile.endswith(".mp4") and Path(outfile).exists():
          print("\nVideo:")
          display(Video(outfile, embed=True, width=480))
  except Exception:
      pass

[+] Leído file3.lol: 205704 bytes
[*] Heurística: parece Base64
[+] Decodificación Base64 OK, tamaño 154276 bytes
[+] Guardado: file3_decoded.mp4  (detected: MP4 (ftyp))
Salida: file3_decoded.mp4

Video:


In [33]:
# Probando que nuestra decodificacion funciona comparando con una libreria
import base64 as _b
text = "VGhpcyBpcyBhIHRlc3Qh"
assert base64_decode_from_scratch(text) == _b.b64decode(text)
print("OK: coinciden")

OK: coinciden
